### Instalações:

In [ ]:
!pip install requests beautifulsoup4 pandas sqlalchemy

In [ ]:
from urllib.request import Request, urlopen, HTTPError, URLError
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import sqlite3

# URLs e tags de interesse
sites_e_tags = {
    "Festas e Shows": {
        "url": "https://www.sympla.com.br/eventos/show-musica-festa",
        "tags": ['h3.EventCardstyle__EventTitle-sc-1rkzctc-7', 'div.EventCardstyle__EventDate-sc-1rkzctc-6', 'div.EventCardstyle__EventLocation-sc-1rkzctc-8']
    },
    "Teatro e Espetáculos": {
        "url": "https://www.sympla.com.br/eventos/teatro-espetaculo",
        "tags": ['h3.EventCardstyle__EventTitle-sc-1rkzctc-7', 'div.EventCardstyle__EventDate-sc-1rkzctc-6', 'div.EventCardstyle__EventLocation-sc-1rkzctc-8']
    },
    "Passeios e Tours": {
        "url": "https://www.sympla.com.br/eventos/experiencias",
        "tags": ['h3.EventCardstyle__EventTitle-sc-1rkzctc-7', 'div.EventCardstyle__EventDate-sc-1rkzctc-6', 'div.EventCardstyle__EventLocation-sc-1rkzctc-8']
    }
}

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36"}

todos_os_eventos = []  # Lista para armazenar todos os eventos de todos os sites
id_evento = 1  # Contador de IDs

for nome_site, dados in sites_e_tags.items():
    url = dados['url']
    tags = dados['tags']

    req = Request(url, headers=headers)
    try:
        response = urlopen(req)
        html = response.read()
        soup = BeautifulSoup(html, 'html.parser')

        # Encontrando os cards dos eventos
        cards_eventos = soup.find_all('div', class_='EventCardstyle__EventCardContainer-sc-1rkzctc-0')

        for card in cards_eventos:
            nome = card.select_one(tags[0]).get_text(strip=True) if card.select_one(tags[0]) else "Nome não disponível"
            data = card.select_one(tags[1]).get_text(strip=True) if card.select_one(tags[1]) else "Data não disponível"
            local = card.select_one(tags[2]).get_text(strip=True) if card.select_one(tags[2]) else "Local não disponível"

            todos_os_eventos.append({
                'ID': id_evento,
                'Nome': nome,
                'Data': data,
                'Local': local,
                'Categoria': nome_site  # Adicionando a categoria
            })
            id_evento += 1

    except HTTPError as e:
        print(f"Erro HTTP em {nome_site}: {e.reason}")
    except URLError as e:
        print(f"Erro URL em {nome_site}: {e.reason}")

# Criando um DataFrame e salvando em CSV
df = pd.DataFrame(todos_os_eventos)
df.to_csv('todos_os_eventos.csv', index=False)

## Formatar a data e salvar em um novo data frame

In [ ]:
# Carregar o CSV (substitua pelo caminho do seu arquivo)
df = pd.read_csv('todos_os_eventos.csv')

# Dicionário para mapear meses abreviados
meses_abreviados = {'Jan': 1, 'Fev': 2, 'Mar': 3, 'Abr': 4, 'Mai': 5, 'Jun': 6,
                    'Jul': 7, 'Ago': 8, 'Set': 9, 'Out': 10, 'Nov': 11, 'Dez': 12}

# Função para extrair e formatar a data (corrigida)
def extrair_e_formatar_data(texto):
    try:
        _, data_hora = texto.split(', ')  # Ignorar o dia da semana
        data, _ = data_hora.split(' · ')    # Ignorar o horário
        dia, mes_abrev = data.split(' ')
        ano = '2024'  # Definir o ano (você pode ajustar isso)
        mes = meses_abreviados[mes_abrev]
        data_str = f'{dia}/{mes}/{ano}'
        return pd.to_datetime(data_str, format='%d/%m/%Y').strftime('%d/%m/%Y')
    except (ValueError, IndexError, KeyError):
        # Extrair dia e mês abreviado
        dia = texto[:2]
        mes_abrev = texto[3:6]

        # Verificar se são valores numéricos e alfabéticos válidos
        if not dia.isdigit() or not mes_abrev.isalpha():
            return None

        ano = '2024'  # Definir o ano (você pode ajustar isso)
        mes = meses_abreviados[mes_abrev.title()]  # Converter para título (ex: "Jun" -> "Jun")
        data_str = f'{dia}/{mes}/{ano}'
        return pd.to_datetime(data_str, format='%d/%m/%Y').strftime('%d/%m/%Y')
        # return None  # Retornar None para entradas inválidas

# Aplicar a função e criar a nova coluna
df['Data_formatada'] = df['Data'].astype(str).apply(extrair_e_formatar_data)

# Imprimir as colunas Data e Data_formatada para verificar o resultado
print(df[['Data', 'Data_formatada']])

                    Data Data_formatada
0           16 Nov18 Nov     16/11/2024
1           09 Nov11 Nov     09/11/2024
2    Sab, 24 Ago · 21:00     24/08/2024
3           08 Jun14 Jul     08/06/2024
4           05 Jul14 Jul     05/07/2024
..                   ...            ...
151         06 Jul07 Jul     06/07/2024
152         04 Jun30 Jul     04/06/2024
153         16 Jul19 Jul     16/07/2024
154  Sab, 08 Jun · 11:00     08/06/2024
155  Sab, 29 Jun · 08:00     29/06/2024

[156 rows x 2 columns]


In [ ]:
df_corrigido = df.drop("Data", axis=1)

In [ ]:
df_corrigido = df_corrigido.rename(columns={'Data_formatada': 'Data'})

In [ ]:
df_corrigido.head()

,ID,Nome,Local,Categoria,Data
0,1,Rock the Mountain 2024 : 2º Final de Semana,Parque Municipal Prefeito Paulo Rattes - Petró...,Festas e Shows,16/11/2024
1,2,Rock the Mountain 2024 : 1º Final de Semana,Parque Municipal Prefeito Paulo Rattes - Petró...,Festas e Shows,09/11/2024
2,3,TIM BERNARDES,"Vivo Rio - Rio de Janeiro, RJ",Festas e Shows,24/08/2024
3,4,Cidade Junina - Festança,"Mirante Beagá - Belo Horizonte, MG",Festas e Shows,08/06/2024
4,5,EXPOAGRO GV 2024,"R. João Dias Duarte, 1450 - Governador Valadar...",Festas e Shows,05/07/2024


In [ ]:
df_corrigido.to_csv('todos_os_eventos_sympla.csv', index=False)